# Unsupervised k-means Clustering Algorithm 

**References**
- [Paper | Unsupervised K-Means Clustering Algorithm](https://ieeexplore.ieee.org/document/9072123)
- [GitHub u-k-means python code](https://github.com/scikit-learn/scikit-learn/issues/23967)

In [1]:
# import libraries
import math
import numpy as np
import pandas as pd

## Instantiating custom class for u_k_means

In [2]:
class u_k_means():
    def __init__(self, thres, beta, gamma, rate, t_max):
        self.thres = thres
        self.beta = beta
        self.gamma = gamma
        self.rate = rate
        self.t_max = t_max

    def fit(self, points, y):
        """

        """
        # initializations
        cluster_n = points.shape[0]
        clust_cen = points + 0.0001
        alpha = np.ones((1, cluster_n)) * 1 / cluster_n
        err = 10

        c_history = []

        while cluster_n > 1 and err >= self.thres:
            self.rate = self.rate + 1

            # Step 2: Compute membership
            u = np.zeros((points.shape[0], cluster_n))
            D7 = np.zeros((points.shape[0],0))
            for k in range(cluster_n):
                D1 = np.subtract(points, clust_cen[k, :])
                D2 = np.power(D1, 2)
                D3 = np.sum(D2, 1)
                D4 = D3
                D5 = self.gamma * math.log(alpha[0, k])
                D6 = np.subtract(D4, D5)
                D7 = np.concatenate((D7, D6[:, np.newaxis]),axis=1)

            if self.rate == 1:
                D8 = D7
                np.fill_diagonal(D7, np.nan)
                val = np.min(D7, axis=1)
                idx = np.argmax(D7, axis=1)
                np.fill_diagonal(D7, np.diag(D8))
            else:
                val = np.min(D7, axis=1)
                idx = np.argmax(D7, axis=1)

            for i in range(points.shape[0]):
                u[i, idx[i]] = 1
            
            print(u)
            # Step 3: Compute gamma
            gamma = math.exp(-cluster_n / 450)

            # Step 4: Update alpha
            new_alpha = np.zeros((1, cluster_n))
            for k in range(cluster_n):
                temp1 = np.power(u[:, k], self.beta)
                temp2 = np.sum(temp1)
                temp3 = temp2 / points.shape[0]
                new_alpha[0, k] = temp3
            
            # Step 5: Update beta
            eta = 1 / points.shape[0]
            temp9 = 0
            for k in range(cluster_n):
                temp8 = math.exp(-eta * points.shape[0] * abs(new_alpha[0, k] - alpha[0, k]))
                temp9 = temp9 + temp8
            temp9 = temp9 / cluster_n
            temp10 = 1 - max(np.sum(u, 1) / points.shape[0])
            temp11 = sum(alpha[0, :] * np.log(alpha[0, :]))
            temp12 = temp10 / (-max(alpha[0, :]) * temp11)
            
            new_beta = min(temp9, temp12)
            
            # Step 6: Update number of clusters
            index = np.where(new_alpha <= 1 / points.shape[0])
            
            # ADJUST ALPHA
            adj_alpha = new_alpha
            adj_alpha = np.delete(adj_alpha, index)
            adj_alpha = adj_alpha / sum(adj_alpha)
            new_alpha = adj_alpha
            if new_alpha.shape[0] == 1:
                new_alpha = alpha
                break
            
            # Update NUMBER OF CLUSTER
            new_cluster_n = new_alpha.shape[0]
            
            # ADJUST MEMBERSHIP(U)
            adj_u = u
            adj_u = np.delete(adj_u, index, 1)
            adj_u = adj_u / np.sum(adj_u, 1)[:, None]
            adj_u[np.isnan(adj_u)] = 0
            new_u = adj_u
            
            if self.rate >= 60 and new_cluster_n - cluster_n == 0:
                new_beta = 0
                
            # Update Cluster Centers
            new_clust_cen = []
            for k in range(new_cluster_n):
                temp4 = np.zeros((1, points.shape[1]))
                temp5 = 0
                for i in range(points.shape[0]):
                    temp4 = temp4 + new_u[i, k] * points[i, :]
                    temp5 = temp5 + new_u[i, k]
                new_clust_cen.append(temp4 / temp5)
                # np.concatenate((new_clust_cen, temp4 / temp5), axis=0)
            new_clust_cen = np.array(new_clust_cen)
            print(new_clust_cen) 
            print(new_clust_cen.shape)    
            new_clust_cen[np.isnan(new_clust_cen)] = sum(np.mean(points))
            
            # STEP 8: Convergence criteria
            error = []
            for k in range(new_cluster_n):
                error = np.concatenate((error, np.linalg.norm(new_clust_cen[k, :] - clust_cen[k, :])))
                
            err = max(error)
            
            clust_cen = new_clust_cen
            cluster_n = new_cluster_n
            alpha = new_alpha
            beta = new_beta
            u = new_u
            c_history.append(cluster_n)
            c_history = np.array(c_history)
            # c_history = np.concatenate((c_history, cluster_n))
            
        # Step 9: Cluster labeling
        clust = []
        for i in range(points.shape[0]):
            clust.append(val[i])
            # np.concatenate((clust, idx[i]))
            
        return clust, c_history

## Testing u-k-means with iris dataset

In [3]:
# load dataset
from sklearn import datasets
iris = datasets.load_iris()

In [4]:
# define input data
points = iris.data
y = iris.target
label = y

In [5]:
# define parameters
thres = 0.1
beta = 1
gamma = 1
rate = 0
t_max = 100

In [6]:
# initializations
cluster_n = points.shape[0]
clust_cen = points + 0.0001
alpha = np.ones((1, cluster_n)) * 1 / cluster_n
err = 10

t_max = 100

c_history = []

while cluster_n > 1 and err >= thres:
    rate = rate + 1

    # Step 2: Compute membership
    u = np.zeros((points.shape[0], cluster_n))
    D7 = []
    for k in range(cluster_n):
        D1 = np.subtract(points, clust_cen[k, :])
        D2 = np.power(D1, 2)
        D3 = np.sum(D2, 1)
        D4 = D3
        D5 = gamma * math.log(alpha[0, k])
        D6 = np.subtract(D4, D5)
        D7.append(D6)
    D7 = np.array(D7)

    if rate == 1:
        D8 = D7
        np.fill_diagonal(D7, np.nan)
        val = np.nanmin(D7, axis=1)
        idx = np.argmin(D7, axis=1)
        np.fill_diagonal(D7, np.diag(D8))
    else:
        val = np.min(D7, axis=1)
        idx = np.argmin(D7, axis=1)

    for i in range(points.shape[0]):
        u[i, idx[i]] = 1

    # Step 3: Compute gamma
    gamma = math.exp(-cluster_n / 450)

    # Step 4: Update alpha
    new_alpha = np.zeros((1, cluster_n))
    for k in range(cluster_n):
        temp1 = np.power(u[:, k], beta)
        temp2 = np.sum(temp1)
        temp3 = temp2 / points.shape[0]
        new_alpha[0, k] = temp3

    # Step 5: Update beta
    eta = 1 / points.shape[0]
    temp9 = 0
    for k in range(cluster_n):
        temp8 = math.exp(-eta * points.shape[0] * abs(new_alpha[0, k] - alpha[0, k]))
        temp9 = temp9 + temp8
    temp9 = temp9 / cluster_n
    temp10 = 1 - max(np.sum(u, 1) / points.shape[0])
    temp11 = sum(alpha[0, :] * np.log(alpha[0, :]))
    temp12 = temp10 / (-max(alpha[0, :]) * temp11)

    new_beta = min(temp9, temp12)

    # Step 6: Update number of clusters
    index = np.where(new_alpha <= 1 / points.shape[0])

    # ADJUST ALPHA
    adj_alpha = new_alpha
    adj_alpha = np.delete(adj_alpha, index)
    adj_alpha = adj_alpha / sum(adj_alpha)
    new_alpha = adj_alpha
    if new_alpha.shape[0] == 1:
        new_alpha = alpha
        # break

    # Update NUMBER OF CLUSTER
    new_cluster_n = new_alpha.shape[0]

    # ADJUST MEMBERSHIP(U)
    adj_u = u
    adj_u = np.delete(adj_u, index, 1)
    adj_u = adj_u / np.sum(adj_u, 1)[:, None]
    adj_u[np.isnan(adj_u)] = 0
    new_u = adj_u

    if rate >= 60 and new_cluster_n - cluster_n == 0:
        new_beta = 0

    # Update Cluster Centers
    new_clust_cen = []
    for k in range(new_cluster_n):
        temp4 = np.zeros((1, points.shape[1]))
        temp5 = 0
        for i in range(points.shape[0]):
            temp4 = temp4 + new_u[i, k] * points[i, :]
            temp5 = temp5 + new_u[i, k]
        new_clust_cen.append(temp4 / temp5)
        # new_clust_cen = np.concatenate((new_clust_cen, temp4 / temp5), axis=0)
    new_clust_cen = np.array(new_clust_cen)
    new_clust_cen[np.isnan(new_clust_cen)] = sum(np.mean(points,axis=0))

    # STEP 8: Convergence criteria
    error = []
    for k in range(new_cluster_n):
        error.append(np.linalg.norm(new_clust_cen[k, :] - clust_cen[k, :]))
        # error = np.concatenate((error, np.linalg.norm(new_clust_cen[k, :] - clust_cen[k, :])))
    error = np.array(error)

    err = max(error)

    clust_cen = new_clust_cen
    cluster_n = new_cluster_n
    alpha = new_alpha
    beta = new_beta
    u = new_u
    c_history.append(c_history)
    # c_history = np.array(c_history)
    # c_history = np.concatenate((c_history, cluster_n))

# Step 9: Cluster labeling
clust = []
for i in range(points.shape[0]):
    clust.append(val[i])
    # clust = np.concatenate((clust, idx[i]))

ValueError: max() arg is an empty sequence

In [18]:
gamma

0.7165313105737893

In [ ]:
clust

[3.5902771140860743,
 3.5902771140860743,
 3.5902771140860743,
 3.5902771140860743,
 3.5902771140860743,
 3.5902771140860743,
 3.5902771140860743,
 3.5902771140860743,
 3.5902771140860743,
 3.5902771140860743,
 3.5902771140860743,
 3.5902771140860743,
 3.5902771140860743,
 3.5902771140860743,
 3.5902771140860743,
 3.5902771140860743,
 3.5902771140860743,
 3.5902771140860743,
 3.5902771140860743,
 3.5902771140860743,
 3.5902771140860743,
 3.5902771140860743,
 3.5902771140860743,
 3.5902771140860743,
 3.5902771140860743,
 3.5902771140860743,
 3.5902771140860743,
 3.5902771140860743,
 3.5902771140860743,
 3.5902771140860743,
 3.5902771140860743,
 3.5902771140860743,
 3.5902771140860743,
 3.5902771140860743,
 3.5902771140860743,
 3.5902771140860743,
 3.5902771140860743,
 3.5902771140860743,
 3.5902771140860743,
 3.5902771140860743,
 3.5902771140860743,
 3.5902771140860743,
 3.5902771140860743,
 3.5902771140860743,
 3.5902771140860743,
 3.5902771140860743,
 3.5902771140860743,
 3.5902771140

In [ ]:
clf = u_k_means(
    thres=thres,
    beta=beta,
    gamma=gamma,
    rate=rate,
    t_max=t_max
)

clusters, cluster_history = clf.fit(points=points, y=y)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[[6.65964912 3.07368421 5.30526316 1.8754386 ]]

 [[5.95416667 2.66666667 4.83333333 1.6       ]]

 [[5.43       2.36       3.75       1.1       ]]

 [[5.08275862 3.33275862 1.78103448 0.37758621]]]
(4, 1, 4)


/var/folders/g4/mj9t8ccd7bn74b0c25x8xjth0000gn/T/ipykernel_45969/2820772806.py:92: RuntimeWarning: invalid value encountered in divide
  adj_u = adj_u / np.sum(adj_u, 1)[:, None]


TypeError: 'numpy.float64' object is not iterable